### It is necessary to change path of the database located in the github folder /data

In [ ]:
import numpy as np
import pandas as pd
import os


column_names = ['patient_number', 'age', 'sex', 'adult_BMI', 'child_weight', 'child_height']
dataset = pd.read_csv('../input/respiratory-sound-database/demographic_info.txt', sep=" ", header=None, names =column_names)
dataset['sex'] = dataset['sex'].replace({'F': 1, 'M': 0})


column_names = ['Beginning of respiratory cycle(s)','End of respiratory cycle(s)','Crackles','Wheezes']
patient_dataset = {}

for file in os.listdir("../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files"):
        if file.endswith(".txt"):
            patient_number = file.split("_")[0]
            data = pd.read_csv('../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/'+file, sep="\t", header=None, names = column_names)
            for crackles in data['Crackles']:
                if crackles == 1:
                    if(patient_dataset.get(patient_number+"_crackles") != None):
                        patient_dataset[patient_number+"_crackles"] += 1
                    else:
                        patient_dataset[patient_number+"_crackles"] = 1;
            for wheezes in data['Wheezes']:
                if wheezes == 1:
                    if(patient_dataset.get(patient_number+"_wheezes") != None):
                        patient_dataset[patient_number+"_wheezes"] += 1
                    else:
                        patient_dataset[patient_number+"_wheezes"] = 1;
            
final_patient_dataset = []

new_data = {#'patient_number': [pd.to_numeric(101)], 
               'n_wheezes': [patient_dataset.get(str(101) + ("_wheezes"))],
               'n_crackles': [patient_dataset.get(str(101) + ("_crackles"))]}
new_data = pd.DataFrame(new_data)
final_strings_for_dataset = pd.DataFrame(new_data)


for patient_number in range (102, 227):  
    
    new_data = {#'patient_number': [pd.to_numeric(patient_number)], 
               'n_wheezes': [patient_dataset.get(str(patient_number) + ("_wheezes"))],
               'n_crackles': [patient_dataset.get(str(patient_number) + ("_crackles"))]}
    new_data = pd.DataFrame(new_data)
    final_strings_for_dataset = final_strings_for_dataset.append([new_data],  ignore_index=True)


final_strings_for_dataset.loc[~final_strings_for_dataset.index.duplicated(keep='first')]
dataset.loc[~dataset.index.duplicated(keep='first')]

dataset = pd.concat([dataset, final_strings_for_dataset], axis=1)
dataset = dataset.replace(np.nan, '0')


Y = pd.read_csv('../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv', sep=",", header=None, names=['patient_number', 'diagnosis'])
Y = Y['diagnosis']

'''
['URTI' 'Healthy' 'Asthma' 'COPD' 'LRTI' 'Bronchiectasis' 'Pneumonia' 'Bronchiolitis']
'''

Y = Y.replace('Healthy', 0)
Y = Y.replace('URTI', 1)
Y = Y.replace('Asthma', 1)
Y = Y.replace('COPD', 1)
Y = Y.replace('LRTI', 1)
Y = Y.replace('Bronchiectasis', 1)
Y = Y.replace('Pneumonia', 1)
Y = Y.replace('Bronchiolitis', 1)
Y = Y.replace(np.nan, '0')

Y = pd.DataFrame(Y)
print(dataset)

## SelectKBest algorithm

In [2]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

dataset_new = SelectKBest(chi2, k=2).fit_transform(dataset, Y)
#print(dataset)
#print(dataset_new)

NameError: name 'dataset' is not defined

## K-fold cross validation

In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

features_for_selection = dataset

svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(n_splits=50), scoring='precision')
rfecv.fit(features_for_selection, np.ravel(Y))

print("Optimal number of features : %d" % rfecv.n_features_)
#print(rfecv.support_)
print(rfecv.ranking_)
print(rfecv.grid_scores_)

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.plot(range(1, len(features_for_selection.columns)+1), rfecv.grid_scores_)
plt.show()